In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

filepath = '/kaggle/input/brent-oil-prices/BrentOilPrices.csv'
data_RAW = pd.read_csv(filepath)
#data_RAW.head()
data_RAW.tail()


In [ ]:
# Starting visualization
import matplotlib.pyplot as plt

plt.figure(figsize=(16,9))
plt.plot(data_RAW[['Price']])
plt.axhline(linestyle='dotted', color='r')
plt.xticks(range(0,data_RAW.shape[0],1000),data_RAW['Date'].loc[::1000],rotation=90)
plt.xlabel('Date', fontsize=20)
plt.ylabel('Oil Price (USD)', fontsize=20)
plt.show()

In [ ]:
# Preparing the dataset

price = data_RAW[['Price']]

# Splitting for training and testing:


sequence_length = 22 # 8 => (1234567 -> 8)
test_percentail = 0.05 # 5% de test

data_raw = price.to_numpy()

In [ ]:
#Normalizer:
from sklearn.preprocessing import MinMaxScaler

scaler= MinMaxScaler()
scaler.fit(data_raw)
norm = scaler.transform(data_raw)
#print(norm)
# reconstruct = scaler.inverse_transform(norm)
# print(reconstruct)


In [ ]:
data = []
for i in range(len(norm) - sequence_length):
  data.append(norm[i:i+sequence_length])

data = np.array(data);
test_set_size = int(np.round(test_percentail*data.shape[0]));
train_set_size = data.shape[0] - (test_set_size);
#print(f'test size: {test_set_size}, train size: {train_set_size}')
#>> test size: 427, train size: 8122

i_train = data[:train_set_size,:-1,:]

o_train = data[:train_set_size,-1,:]

i_test = data[train_set_size:,:-1]

o_test = data[train_set_size:,-1,:]


#print(f'{i_train.shape} \n{o_train.shape} \n{i_test.shape} \n{o_test.shape}')

In [ ]:
# Model building
# Models Parameters
import torch
import torch.nn as nn
import time

i_train_tensor = torch.from_numpy(i_train).type(torch.Tensor)
i_test_tensor = torch.from_numpy(i_test).type(torch.Tensor)
o_train_tensor = torch.from_numpy(o_train).type(torch.Tensor)
o_test_tensor = torch.from_numpy(o_test).type(torch.Tensor)

input_dim = 1   # Lista de {sequence_length-1}
hidden_dim = 22 # Memoria interna das RNN/RNR
num_layers = 2  # 
output_dim = 1  # 1 Lista de 1 elemento
num_epochs = 100

learn_rate = 0.01

In [ ]:
# LSTM:

class LSTM(nn.Module):
  def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
    super(LSTM, self).__init__()
    self.hidden_dim = hidden_dim
    self.num_layers = num_layers        
    self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_dim, output_dim)
    
  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
    out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
    out = self.fc(out[:, -1, :]) 
    return out

model_LSTM = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
#criterion = torch.nn.MSELoss(reduction='sum')
#criterion = torch.nn.L1Loss()
optimiser = torch.optim.Adam(model_LSTM.parameters(), lr=learn_rate)
#optimiser = torch.optim.Adagrad(model_LSTM.parameters(), lr=learn_rate)
#optimiser = torch.optim.AdamW(model_LSTM.parameters(), lr=learn_rate)

hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []

for t in range(num_epochs):
  o_train_pred = model_LSTM(i_train_tensor)
  loss = criterion(o_train_pred, o_train_tensor)
  #print("Epoch ", t+1, "Loss: ", loss.item())
  hist[t] = loss.item()
  optimiser.zero_grad()
  loss.backward()
  optimiser.step()
    
training_time = time.time()-start_time
print(f'Training time: {training_time}')


In [ ]:
plt.figure(figsize=(16,9))
plt.plot(hist)
plt.axhline(linestyle='dotted', color='r')
plt.xlabel('Epoch', fontsize=20)
plt.ylabel('Loss', fontsize=20)

plt.show()

In [ ]:
#Results:

train_score1 = criterion(model_LSTM(i_train_tensor).detach(), o_train_tensor)
test_score1 = criterion(model_LSTM(i_test_tensor).detach(), o_test_tensor)

print('Parametros: ',
          f'\n  Arquiteture {model_LSTM}',
          f'\n  Input lookback length: {sequence_length-1}',
          f'\n  Learning Rate: {learn_rate}',
          f'\n  Training/Testing Relation: {(1-test_percentail)*100}%/{100*test_percentail}%',
          f'\n  Loss function: {criterion}')


print('\nResultados',
      f'\n  Training time: {training_time:.2f} seconds',
      f'\n  Training Loss: {train_score1.item()}',
      f'\n  Test Loss: {test_score1.item()}')

In [ ]:
plt.figure(figsize=(16,9))
plt.plot(scaler.inverse_transform(model_LSTM(i_test_tensor).detach().numpy()))
plt.plot(scaler.inverse_transform(o_test))
plt.axhline(linestyle='dotted', color='r')
plt.xlabel('Date', fontsize=20)
plt.ylabel('Oil Price (USD)', fontsize=20)
plt.legend(['LSTM', 'Real'])

plt.show()

In [ ]:
# GRU:

class GRU(nn.Module):
  def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
    super(GRU, self).__init__()
    self.hidden_dim = hidden_dim
    self.num_layers = num_layers       
    self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
    out, (hn) = self.gru(x, (h0.detach()))
    out = self.fc(out[:, -1, :]) 
    return out

model_GRU = GRU(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
criterion = torch.nn.MSELoss(reduction='mean')
#criterion = torch.nn.MSELoss(reduction='sum')
#criterion = torch.nn.L1Loss()
optimiser = torch.optim.Adam(model_GRU.parameters(), lr=learn_rate)
#optimiser = torch.optim.Adagrad(model_GRU.parameters(), lr=learn_rate)
#optimiser = torch.optim.AdamW(model_GRU.parameters(), lr=learn_rate)

hist2 = np.zeros(num_epochs)
start_time = time.time()
gru = []

for t in range(num_epochs):
  o_train_out = model_GRU(i_train_tensor)
  loss = criterion(o_train_out, o_train_tensor)
  #print("Epoch ", t+1, "Loss: ", loss.item())
  hist2[t] = loss.item()
  optimiser.zero_grad()
  loss.backward()
  optimiser.step()

training_time_GRU = time.time()-start_time    
print(f'Training time: {training_time_GRU}')



In [ ]:
plt.figure(figsize=(16,9))
plt.plot(hist2)
plt.axhline(linestyle='dotted', color='r')
plt.xlabel('Epoch', fontsize=20)
plt.ylabel('Loss', fontsize=20)

plt.show()

In [ ]:
train_score2 = criterion(model_GRU(i_train_tensor).detach(), o_train_tensor)
test_score2 = criterion(model_GRU(i_test_tensor).detach(), o_test_tensor)


print('Parametros: ',
          f'\n  Arquiteture {model_GRU}',
          f'\n  Input lookback length: {sequence_length-1}',
          f'\n  Learning Rate: {learn_rate}',
          f'\n  Training/Testing Relation: {(1-test_percentail)*100}%/{100*test_percentail}%',
          f'\n  Loss function: {criterion}')


print('\nResultados',
      f'\n  Training time: {training_time_GRU:.2f}  seconds',
      f'\n  Training Loss: {train_score2.item()}',
      f'\n  Test Loss: {test_score2.item()}')



In [ ]:
plt.figure(figsize=(16,9))
plt.plot(scaler.inverse_transform(model_GRU(i_test_tensor).detach().numpy()))
plt.plot(scaler.inverse_transform(o_test))
plt.axhline(linestyle='dotted', color='r')
plt.xlabel('Date', fontsize=20)
plt.ylabel('Oil Price (USD)', fontsize=20)
plt.legend(['GRU', 'Real'])

plt.show()


In [ ]:
# Comparing LSTM e GRU:

plt.figure(figsize=(16,9))
plt.plot(scaler.inverse_transform(o_test))
plt.plot(scaler.inverse_transform(model_GRU(i_test_tensor).detach().numpy()))
plt.plot(scaler.inverse_transform(model_LSTM(i_test_tensor).detach().numpy()))
plt.axhline(linestyle='dotted', color='r')
#plt.xticks(range(428),pred_y,rotation=90)
plt.xlabel('Date', fontsize=20)
plt.ylabel('Oil Price (USD)', fontsize=20)

plt.legend(['Real', 'GRU', 'LSTM'])
plt.show()

In [ ]:
# Loss function in training (LSTM and GRU):

plt.figure(figsize=(16,9))
plt.plot(hist)
plt.plot(hist2)
plt.axhline(linestyle='dotted', color='r')
plt.xlabel('Epoch', fontsize=20)
plt.ylabel('Loss', fontsize=20)

plt.legend(['LSTM', 'GRU'])
plt.show()